<img src="NotebookAddons/blackboard-banner.jpg" width="100%" />
<font face="Calibri">
<br>
<font size="7"> <b> GEOS 657: Microwave Remote Sensing <b> </font>

<font size="5"> <b>Lab 7: Deep Learning in Earth Observation: Demo Exercise </b> </font>

<br>
<font size="4"> <b> Lichao Mou, German Aerospace Center; Xiaoxiang Zhu, German Aerospace Center & Technical University Munich </b> <br>
</font>

<img src="NotebookAddons/dlr-logo-png-transparent.png" width="170" align="right" border="2"/> <font size="3"> This Lab introduces you to the basic concepts of Deep Learning in Earth Observation. Specifically, it uses the simple example of learning the temporal pattern of a cosine curve to demonstrate the concepts of Recurrent Neural Networks (RNNs). The lab let's you experiment with several hyper-parameters needed for training Deep Learning Networks such as RNNs, CNNs, or similar.
    
We will again use a **Jupyter Notebook** framework implemented within the Amazon Web Services (AWS) cloud to work on this exercise. This Lab is part of the UAF course <a href="https://radar.community.uaf.edu/" target="_blank">GEOS 657: Microwave Remote Sensing</a>. It will introduce the following data analysis concepts:

- How to set up a recurrent deep network within the Python-based <i>keras/tensorflow</i> environment
- How to create an LSTM (long-term/short-term memory) recurrent network 
- How to optimize hyper-parameters when training a deep neural network
</font>

<font size="4"> <font color='rgba(200,0,0,0.2)'> <b>There are no Homework assignments associated with this Notebook </b> </font>
</font>
<br>
<hr>

# Predict a cosine wave using RNNs

* A simple tutorial on LSTM and GRU to perdict a trigonometric wave.

* Data noise can be added to test the robustness of the model.

* Hyperparamters of the RNNs can be tweaked


## Import dependencies


In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, GRU, TimeDistributed
from keras.optimizers import RMSprop


## Get cosine data

Data to train and evaluate the RNN:

* Start, end and step define the range of the data series.
* Sequence length defines the series to look back to train the model
* Noisy data can be added to make the training data imperfect.

In [ ]:
def cosine_data(start, end, step, sequence_length, noise_level=0):
    '''
    Define training data
    :param start: Starting point
    :param end: End point
    :param step: Steps between points
    :param sequence_length: Number of steps to backpropagate through time
    :param noise_level: add noise to create imperfect data
    :return: X,Y data
    '''
    t = np.arange(start, end, step)
    cosine = np.cos(2 * np.pi * t) + noise_level*np.random.normal(0,1, np.shape(t))
    cosine = cosine.reshape((cosine.shape[0], 1))
    
    dX, dY = [], []
    for i in range(len(cosine) - 2*sequence_length):
        dX.append(cosine[i:i + sequence_length])
        dY.append(cosine[i + sequence_length:i + 2*sequence_length])
    dataX = np.array(dX)
    dataY = np.array(dY)
    return dataX, dataY


## Create a LSTM model

* Linear activation
* Loss in mean squared error


In [ ]:
def LSTM_(hidden_neurons, feature_count, learning_rate):
    '''
    Define a LSTM model
    :param hidden_neurons: number of neurons to train a GRU network
    :param feature_count: number of features to predict
    :param learning_rate
    :return: Return model for training
    '''

    model = Sequential()
    model.add(LSTM(input_dim=feature_count, output_dim=hidden_neurons, return_sequences=True))
    model.add(TimeDistributed(Dense(feature_count)))
    model.add(Activation('linear'))
    optimizer = RMSprop(lr=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])
    return model


## Create a GRU model

* Linear activation
* Get loss using a mean squared error



In [ ]:
def GRU_(hidden_neurons, feature_count, learning_rate):
    '''
    Define a GRU model
    :param hidden_neurons: number of neurons to train a GRU network
    :param feature_count: number of features to predict
    :param learning_rate 
    :return: Return model for training
    '''

    model = Sequential()
    model.add(GRU(input_dim=feature_count, output_dim=hidden_neurons, return_sequences=True))
    model.add(TimeDistributed(Dense(feature_count)))
    model.add(Activation('linear'))
    optimizer = RMSprop(lr=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])
    return model


## Function to train a RNN model


In [ ]:
def train_cosine(model, dataX, dataY, batch_size, epoch_count):
    '''
    :param model: load RNN model
    :param dataX: X cosine train data
    :param dataY: Y cosine train data
    :param batch_size: Number of samples that going to be propagated through the network
    :param epoch_count: Number of time dataset is processed
    :return: training and validation loss
    '''
    history = model.fit(dataX, dataY, batch_size=batch_size, epochs=epoch_count, validation_split=0.05)
    loss_history = history.history['loss']
    loss_history = np.array(loss_history)
    #np.savetxt("loss_history.txt", numpy_loss_history, delimiter=",")
    val_loss_history = history.history['val_loss']
    val_loss_history = np.array(val_loss_history)
    #np.savetxt("val_loss_history.txt", numpy_loss_history, delimiter=",")
    loss = history.history['loss']
    loss_val = history.history['val_loss']
    plt.rcParams.update({'font.size': 18})
    fig = plt.figure(figsize=(8,7))
    ax = fig.add_subplot(1,1,1)
    plt.plot(loss)
    plt.plot(loss_val)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper right')
    plt.show()

    return loss_history, val_loss_history


## Run RNN code


In [ ]:
def test_cosine(EPOCHS, noise_level=0.3, sequence_length=100, learning_rate=1e-3, batch_size=16, nb_units=32, plot_results=False):
    '''
    Fuction to run a RNN at the specified parameters
    :param EPOCHS: Number of Epochs
    :param noise_level: Added noise level in training data
    :param sequence_length: Sequence length available to train a RNN
    :param learning_rate
    :param batch_size
    :param nb_units
    :param plot_results: Boolean if results should be plotted
    :return: Loss and Plot
    '''
    dataX, dataY = cosine_data(0.0, 10, 0.02, sequence_length, noise_level)  #4.0
    # create and fit the LSTM network
    print('creating model...')

    # Choose RNN to train
    model = LSTM_(nb_units, 1, learning_rate)
    #model = GRU_(nb_units, 1, learning_rate)

    # Train RNN model
    tr_loss, val_loss = train_cosine(model, dataX, dataY, batch_size, EPOCHS)

    # now test
    dataX1, dataY1 = cosine_data(15.0, 21.0, 0.02, sequence_length)
    predict = model.predict(dataX1)

    # now plot
    nan_array = np.empty((sequence_length - 1))
    nan_array.fill(np.nan)
    nan_array2 = np.empty(sequence_length)
    nan_array2.fill(np.nan)
    ind = np.arange(2*sequence_length)


            
    if plot_results == True:
        plt.rcParams.update({'font.size': 18})
        fig = plt.figure(figsize=(8,7))
        ax = fig.add_subplot(1,1,1)
        #fig, ax = plt.subplots()
        for i in range(0, sequence_length, sequence_length):
            forecasts = np.concatenate((nan_array, dataX1[i, -1:, 0], predict[i, :, 0]))
            ground_truth = np.concatenate((nan_array, dataX1[i, -1:, 0], dataY1[i, :, 0]))
            network_input = np.concatenate((dataX[i, :, 0], nan_array2))

            ax.plot(ind, network_input, 'b-x', label='Network input')
            ax.plot(ind, forecasts, 'r-x', label='Many to many model forecast')
            ax.plot(ind, ground_truth, 'g-x', label='Ground truth')

            plt.xlabel('t')
            plt.ylabel('cos(t)')
            plt.title('Cosine Many to Many Forecast')
            plt.legend(loc='best')
            #plt.savefig(os.getcwd() + "cosine_wave" + str(i) + '.png')
            plt.show()
    return tr_loss, val_loss


In [ ]:
def main():
    
    nb_epochs = 10
    noise_level = 0.0
    sequence_length = 100
    '''
    learning_rate = 1e-3  # 1e-1, 1e-2, 1e-3, 1e-4, 1e-5
    batch_size = 16  # 1, 2, 4, 8, 16
    nb_units = 32  # 8, 16, 32, 64, 128

    _, _ = test_cosine(EPOCHS = nb_epochs, noise_level =noise_level, 
                       plot_results=True)
    '''
    # try with different noise
    '''
    noise_level_range = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
    for nl in noise_level_range:
        tr_loss, val_loss = test_cosine(EPOCHS = nb_epochs, noise_level=nl)
        print('tr_loss:', tr_loss)
        print('val_loss:', val_loss)
    '''
    
    # hyperparameter: 1) lr
    learning_rate_range = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    for lr in learning_rate_range:
        tr_loss, val_loss = test_cosine(EPOCHS = nb_epochs, learning_rate=lr, plot_results=True)
        print('tr_loss:', tr_loss)
        print('val_loss:', val_loss)
    
    '''
    # hyperparameter: 2) batch size
    batch_size_range = [2, 4, 8, 16, 32]
    for bs in batch_size_range:
        tr_loss, val_loss = test_cosine(EPOCHS = nb_epochs, batch_size=bs, plot_results=True)
        print('tr_loss:', tr_loss)
        print('val_loss:', val_loss)
    '''
    '''
    # hyperparameter: 3) sequence length
    sequence_length_range = [20, 50, 100, 200, 500]
    for sl in sequence_length_range:
        tr_loss, val_loss = test_cosine(EPOCHS = nb_epochs, sequence_length=sl, plot_results=True)
        print('tr_loss:', tr_loss)
        print('val_loss:', val_loss)
    '''
    '''
    # hyperparameter: 4) nb_units
    nb_units_range = [8, 16, 32, 64, 128]
    for nu in nb_units_range:
        tr_loss, val_loss = test_cosine(EPOCHS = nb_epochs, nb_units=nu, plot_results=True)
        print('tr_loss:', tr_loss)
        print('val_loss:', val_loss)
    '''

    return 1


In [ ]:

if __name__ == "__main__":
    sys.exit(main())